In [ ]:
import os 
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [24]:

xml_list=glob('./data_images/*.xml')
xml_list=list(map(lambda x:x.replace('\\','/'),xml_list))

In [39]:

def extract_text(filename):
            tree=et.parse(filename) 
            root=tree.getroot()
            image_name= root.find('filename').text 
    
            width=root.find('size').find('width').text
            height=root.find('size').find('height').text
            objs =root.findall('object')
            parser = []
            for obj in objs:
                            name = obj.find('name').text
                            bndbox= obj.find('bndbox')
                            xmin =bndbox.find('xmin').text
                            xmax = bndbox.find('xmax').text
                            ymin =bndbox.find('ymin').text
                            ymax= bndbox.find('ymax').text
                            parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])
                        
            return parser


In [40]:
parser_all=list(map(extract_text,xml_list))

In [41]:
data = reduce(lambda x,y : x+y,parser_all)

In [53]:
df=pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])


In [54]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,000001.jpg,1024,657,car,14,301,335,522
1,000001.jpg,1024,657,car,269,571,345,489
2,000001.jpg,1024,657,car,502,798,342,450
3,000001.jpg,1024,657,car,709,1009,333,438
4,000002.jpg,800,600,car,41,768,240,497


In [55]:
#type conversions

cols=['width','height','xmin','xmax','ymin','ymax']
df[cols]=df[cols].astype(int)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15663 entries, 0 to 15662
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  15663 non-null  object
 1   width     15663 non-null  int64 
 2   height    15663 non-null  int64 
 3   name      15663 non-null  object
 4   xmin      15663 non-null  int64 
 5   xmax      15663 non-null  int64 
 6   ymin      15663 non-null  int64 
 7   ymax      15663 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 979.1+ KB


In [57]:
df['center_x']=((df['xmin'] + df['xmax'])/2)/df['width']
df['center_y']=((df['ymin'] + df['ymax'])/2)/df['height']

df['w']=(df['xmax']-df['xmin'])/df['width']
df['h']=(df['ymax']-df['ymin'])/df['height']

In [58]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,000001.jpg,1024,657,car,14,301,335,522,0.153809,0.652207,0.280273,0.284627
1,000001.jpg,1024,657,car,269,571,345,489,0.410156,0.634703,0.294922,0.219178
2,000001.jpg,1024,657,car,502,798,342,450,0.634766,0.602740,0.289062,0.164384
3,000001.jpg,1024,657,car,709,1009,333,438,0.838867,0.586758,0.292969,0.159817
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333


In [59]:
images=df['filename'].unique()

In [65]:
img_df=pd.DataFrame(images,columns=['filename'])
img_df.head()
img_train=tuple(img_df.sample(frac=0.8)['filename'])
img_test=tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [66]:
len(img_train),len(img_test)

(4010, 1002)

In [67]:
train_df=df.query(f'filename in {img_train}')
test_df=df.query(f'filename in {img_test}')

In [77]:

def label_encoding(x):
        labels = {'person': 0, 'car':1, 'chair': 2, 'bottle':3, 'pottedplant':4, 'bird':5, 'dog':6,
        'sofa':7, 'bicycle':8, 'horse':9, 'boat':10, 'motorbike':11, 'cat':12, 'tvmonitor' :13, 'cow':14, 
                  'sheep':15, 'aeroplane':16, 'train':17, 'diningtable' :18, 'bus':19}
        return labels [x]

In [78]:
 train_df['id'] = train_df['name'].apply(label_encoding)
 test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\praty\AppData\Local\Temp\ipykernel_19976\179443206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\praty\AppData\Local\Temp\ipykernel_19976\179443206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [79]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
4,000002.jpg,800,600,car,41,768,240,497,0.505625,0.614167,0.908750,0.428333,1
5,000002.jpg,800,600,car,533,722,236,299,0.784375,0.445833,0.236250,0.105000,1
6,000007.jpg,500,333,car,141,500,50,330,0.641000,0.570571,0.718000,0.840841,1
11,000012.jpg,500,333,car,156,351,97,270,0.507000,0.551051,0.390000,0.519520,1
13,000017.jpg,480,364,person,185,279,62,199,0.483333,0.358516,0.195833,0.376374,0


In [80]:
import os
from shutil import move

In [81]:
train_folder='data_images/train'
test_folder='data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [85]:
cols=['filename','id','center_x','center_y','w']
groupby_obj_train=train_df[cols].groupby('filename')
groupby_obj_test=test_df[cols].groupby('filename')

In [84]:
def save_data(filename,folder_path,group_obj) :
    src=os.path.join('data_images',filename)
    dst=os.path.join(folder_path,filename)
    move(src,dst)
    text_filename=os.path.join(folder_path,os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv('text_filename',sep=' ',index=False,header=False)

    

In [87]:
filename_series=pd.Series(groupby_object_train.groups.keys())

In [88]:
filename_series

0       000002.jpg
1       000007.jpg
2       000012.jpg
3       000017.jpg
4       000019.jpg
           ...    
4005    009949.jpg
4006    009950.jpg
4007    009954.jpg
4008    009958.jpg
4009    009959.jpg
Length: 4010, dtype: object

In [89]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

0       None
1       None
2       None
3       None
4       None
        ... 
4005    None
4006    None
4007    None
4008    None
4009    None
Length: 4010, dtype: object

In [90]:
filename_series_test=pd.Series(groupby_object_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0       None
1       None
2       None
3       None
4       None
        ... 
997     None
998     None
999     None
1000    None
1001    None
Length: 1002, dtype: object